In [1]:
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
directory = "..//textExtractor//test_DataOutput"
data=''
for filename in os.listdir(directory):
    f=open(os.path.join(directory, filename),"r+",encoding="UTF-8")
    data=data+f.read()
len(data)

582207

In [4]:
cleaned = re.sub(r'\W+', ' ', data).lower()
tokens = word_tokenize(cleaned)
train_len = 3+1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [5]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape
#print(train_targets[0])

(92315, 3)

In [6]:
from keras.models import Sequential, load_model
model = load_model("ModelNextWord.h5")

In [8]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)


[462, 2605, 85] [[ 462 2605   85]]
Next word suggestion: en
Next word suggestion: pudiéramos
Next word suggestion: por


In [10]:
input_text = input().strip().lower()
print(type(input_text))
full_text = input_text
for i in range(50):
    encoded_text = tokenizer.texts_to_sequences([input_text])[0]
    pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
    print(encoded_text, pad_encoded)
    next_word_index=(model.predict(pad_encoded)[0]).argsort()[-5:][::-1][0]
    next_word=tokenizer.index_word[next_word_index]
    print(next_word)
    input_text=input_text.split()
    input=input_text[1:]
    input=' '.join(input)
    input=input+' '+next_word
    full_text = full_text+next_word
    

TypeError: 'str' object is not callable